# Ánalise Comparativa

* **Importando Bibliotecas**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, ShuffleSplit, KFold, cross_validate

#### **Leitura dos Dados**

In [2]:
df= pd.read_csv("../data/raw/wines_SPA.csv")
df.head()

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity
0,Teso La Monja,Tinto,2013,4.9,58,Espana,Toro,995.00,Toro Red,5.0,3.0
1,Artadi,Vina El Pison,2018,4.9,31,Espana,Vino de Espana,313.50,Tempranillo,4.0,2.0
2,Vega Sicilia,Unico,2009,4.8,1793,Espana,Ribera del Duero,324.95,Ribera Del Duero Red,5.0,3.0
3,Vega Sicilia,Unico,1999,4.8,1705,Espana,Ribera del Duero,692.96,Ribera Del Duero Red,5.0,3.0
4,Vega Sicilia,Unico,1996,4.8,1309,Espana,Ribera del Duero,778.06,Ribera Del Duero Red,5.0,3.0


In [3]:
df_dict = pd.read_csv('../data/external/dictionary.csv')
df_dict

,variavel,descricao,tipo,subtipo
0,winery,Nome da vinícola,qualitativa,nominal
1,wine,Nome do vinho,qualitativa,nominal
2,year,Ano em que as uvas foram colhidas,quantitativa,discreta
3,rating,Avaliação média dada ao vinho pelos usuários [...,quantitativa,contínua
4,num_reviews,Número de usuários que avaliaram o vinho,quantitativa,discreta
5,country,País de origem [Espanha],inútil,inútil
6,region,Região do vinho,qualitativa,nominal
7,price,Preço em euros [€],quantitativa,contínua
8,type,Variedade de vinho,qualitativa,nominal
9,body,"Pontuação de corpo, definida como a riqueza e ...",qualitativa,ordinal


#### **Renomeando as colunas**

In [4]:
mapeamento_colunas = {
    'winery': 'vinícola',
    'wine': 'vinho',
    'year': 'ano',
    'rating': 'avaliação',
    'num_reviews': 'num_avaliações',
    'country': 'país',
    'region': 'região',
    'price': 'preço',
    'type': 'tipo',
    'body': 'corpo',
    'acidity': 'acidez'
}

In [5]:
def renomear_colunas(df, mapping):
    df.rename(columns=mapping, inplace=True)
    return df

In [6]:
df_dict['variavel'] = df_dict['variavel'].replace(mapeamento_colunas)
df = renomear_colunas(df, mapeamento_colunas)

## Tratamento e Transformação de dados

#### **Verificando os Valores Faltantes e Dados Duplicados**

In [7]:
print(df.duplicated().sum())

5452


In [8]:
df = df.drop(df[df.duplicated()].index, axis=0)
df.reset_index()
print(f'Number of Duplicates: {df.duplicated().sum()}')

Number of Duplicates: 0


In [9]:
df['ano'] = df['ano'].replace('N.V.', np.nan)

In [10]:
print(f'Verificando valores faltantes : \n\n{df.isna().sum()}')

Verificando valores faltantes : 

vinícola            0
vinho               0
ano                72
avaliação           0
num_avaliações      0
país                0
região              0
preço               0
tipo              106
corpo             271
acidez            271
dtype: int64


#### **Verificando os Valores Discrepantes**

In [11]:
# Função para identificar outliers usando o IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # Primeiro quartil
    Q3 = df[column].quantile(0.75)  # Terceiro quartil
    IQR = Q3 - Q1                     # Intervalo Interquartil
    
    lower_bound = Q1 - 1.5 * IQR      # Limite inferior
    upper_bound = Q3 + 1.5 * IQR      # Limite superior
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

In [12]:
# Detectando outliers apenas para colunas numéricas
outliers_summary = []

for col in df.select_dtypes(include=[np.number]).columns:  # Filtra apenas colunas numéricas
    outliers = detect_outliers_iqr(df, col)
    if not outliers.empty:
        outliers_summary.append({'Coluna': col, 'Quantidade de Outliers': len(outliers)})

outliers_df = pd.DataFrame(outliers_summary)

print(outliers_df)

           Coluna  Quantidade de Outliers
0       avaliação                       2
1  num_avaliações                     242
2           preço                     245
3           corpo                      34
4          acidez                     105


# Pré-Processamento dos Dados

#### **Definição das Variáveis do Modelo de Previsão**

In [13]:
# Variável alvo
target_variable = 'preço'

# Variáveis inúteis
useless_variables = (
    df_dict
    .query("tipo == 'inútil'")
    .variavel
    .to_list()
)

# Variáveis nominais
nominal_variables = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis ordinais
ordinal_variables = (
    df_dict
    .query("subtipo == 'ordinal' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis contínuas
continuous_variables = (
    df_dict
    .query("subtipo == 'contínua' and variavel != @target_variable")
    .variavel
    .to_list()
)

# Variáveis discretas
discrete_variables = (
    df_dict
    .query("subtipo == 'discreta' and variavel != @target_variable")
    .variavel
    .to_list()
)

In [14]:
X = df.drop(columns=[target_variable] + useless_variables)
y = df[target_variable]

In [15]:
# Exibindo as categorias de variáveis
print("Nominal Variables:", nominal_variables)
print("Ordinal Variables:", ordinal_variables)
print("Continuous Variables:", continuous_variables)
print("Discrete Variables:", discrete_variables)

Nominal Variables: ['vinícola', 'vinho', 'região', 'tipo']
Ordinal Variables: ['corpo', 'acidez']
Continuous Variables: ['avaliação']
Discrete Variables: ['ano', 'num_avaliações']


#### **Pré-processamento das Variáveis: Imputação e Codificação**

In [16]:
# Nominal variables: imputação de valores faltantes e codificação one-hot
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="most_frequent")),  # Preenchimento com o valor mais frequente
    ("encoding", OneHotEncoder(sparse_output=False, handle_unknown="infrequent_if_exist"))  # Codificação One-Hot
])

# Ordinal variables: imputação de valores faltantes e codificação ordinal
ordinal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento com a mediana
    ("encoding", OrdinalEncoder())  # Codificação ordinal
])

# Continuous variables: imputação de valores faltantes e normalização
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="mean")),  # Preenchimento com a média
    ("normalization", StandardScaler())  # Normalização
])

# Discrete variables: imputação de valores faltantes e normalização
discrete_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy="median")),  # Preenchimento baseado em K vizinhos mais próximos
    ("normalization", StandardScaler())  # Normalização
])

In [17]:
# Criação do ColumnTransformer para processar cada tipo de variável
preprocessor = ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),   
    ("ordinal", ordinal_preprocessor, ordinal_variables),   
    ("continuous", continuous_preprocessor, continuous_variables),  
    ("discrete", discrete_preprocessor, discrete_variables), 
])

In [18]:
preprocessor

ColumnTransformer(transformers=[('nominal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoding',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False))]),
                                 ['vinícola', 'vinho', 'região', 'tipo']),
                                ('ordinal',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('encoding',
                                                  OrdinalEncoder())]),
                                 ['corpo', 'acidez']),
                                ('continuous',
                                 Pipeline(steps=[('missing', SimpleImputer()),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['avaliação']),
                                ('discrete',
                                 Pipeline(steps=[('missing',
                                                  SimpleImputer(strategy='median')),
                                                 ('normalization',
                                                  StandardScaler())]),
                                 ['ano', 'num_avaliações'])])

# **Avaliação e Comparação de Modelos**

#### **Validação Cruzada e Métricas de Desempenho**

In [19]:
modelos = [DummyRegressor(strategy="mean"), LinearRegression(), SVR()]

In [20]:
#Modelo que deu Erro

#modelos = [, KNeighborsRegressor(n_neighbors=5)]

In [21]:
metricas = [
    'neg_mean_absolute_error',
    'neg_mean_squared_error',
    'neg_mean_absolute_percentage_error',
    'r2',
]

monte_carlo = ShuffleSplit(n_splits=10, test_size=.2, random_state=42)

In [22]:
results_total = None
for modelo in modelos:
    model_name = modelo.__class__.__name__
    print(f"rodando para o modelo: {model_name}")
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('Modelo', modelo),
    ])

    scores = cross_validate(
        approach, X, y,
        scoring=metricas, 
        cv=monte_carlo
    )
    results_model = pd.DataFrame(scores)
    results_model['Modelo'] = model_name
    if results_total is None:
        results_total = results_model
    else:
        results_total = pd.concat([results_total, results_model])

rodando para o modelo: DummyRegressor
rodando para o modelo: LinearRegression
rodando para o modelo: SVR


In [23]:

results_total.groupby('Modelo').agg(['mean', 'std']).T

Modelo                                        DummyRegressor  \
fit_time                                mean        0.035326   
                                        std         0.016493   
score_time                              mean        0.016891   
                                        std         0.004411   
test_neg_mean_absolute_error            mean     -135.416637   
                                        std         9.333180   
test_neg_mean_squared_error             mean   -79117.891312   
                                        std     21602.469130   
test_neg_mean_absolute_percentage_error mean       -2.359083   
                                        std         0.131802   
test_r2                                 mean       -0.002589   
                                        std         0.002716   

Modelo                                        LinearRegression           SVR  
fit_time                                mean      8.507029e-01      1.883123  
                                        std       1.517255e-01      0.061945  
score_time                              mean      1.711240e-02      0.673172  
                                        std       3.624818e-03      0.010609  
test_neg_mean_absolute_error            mean     -3.700495e+13    -89.372951  
                                        std       4.289053e+13     10.849919  
test_neg_mean_squared_error             mean     -3.113273e+28 -76280.883168  
                                        std       6.284411e+28  22156.517025  
test_neg_mean_absolute_percentage_error mean     -1.332749e+12     -0.506741  
                                        std       1.684055e+12      0.022697  
test_r2                                 mean     -3.140074e+23      0.038300  
                                        std       5.561496e+23      0.020349